In [1]:
import numpy as np
import json
import os
import matplotlib.pyplot as plt
import shutil
import matplotlib.image as mpimg
import skimage
import glob
import pandas as pd
import datetime

## Custom imports 
import utils.common as common
from barchart import draw_and_save_line_graph


/home/alpha/virtualmachines/virtualenvs/py_3-6-5_2018-11-21/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def sort_evaluate_data(stats, sort_on, f1, f2):
    iArr = {}
    assort = { "classname":[] }
    assort[sort_on] = []
    assort[f1] = []
    assort[f2] = []

    
    for i in range(len(stats[sort_on])):
      iArr[stats[sort_on][i]] = { "classname": stats["classname"][i] }
      iArr[stats[sort_on][i]][sort_on] = stats[sort_on][i]
      iArr[stats[sort_on][i]][f1] = stats[f1][i]
      iArr[stats[sort_on][i]][f2] = stats[f2][i]
  
    
    sorted_val = sorted(iArr,reverse=True)
    for i in range(len(sorted_val)):
      assort["classname"].append(iArr[sorted_val[i]]["classname"])
      assort[sort_on].append(iArr[sorted_val[i]][sort_on])
      assort[f1].append(iArr[sorted_val[i]][f1])
      assort[f2].append(iArr[sorted_val[i]][f2])

    return assort


def load_evaluate_data(evaluate_data_filepath):
    if os.path.isfile(evaluate_data_filepath):
        with open(evaluate_data_filepath) as json_file:  
            evaluate_json_data = json.load(json_file)
    else:
        print("File Does Not Exists! evaluate_data_filepath: {}".format(evaluate_data_filepath))
    
    return evaluate_json_data


def get_stats(evaluate_data, username='', iou=0, evaluate_dir='', on=''):
    """Extracting desired values from original json in stats ignoring BG"""
    evaluate_report = {
        'username':username
        ,'iou':iou
        ,'evaluate_dir':evaluate_dir
        ,'on':on
        ,'classname':[]
        ,'gt_total_annotation':[]
        ,'pred_match_total_annotation':[]
        ,'avg_precision':[]
        ,'avg_recall':[]
        ,'mAP':[]
    }

    for classname in evaluate_data.keys():
        print("classname: {}".format(classname))
        if classname=='BG':
            continue
        evaluate_report['classname'].append(classname)
        evaluate_report['gt_total_annotation'].append((evaluate_data[classname]['gt_total_annotation']))
        evaluate_report['pred_match_total_annotation'].append(evaluate_data[classname]['pred_match_total_annotation'])
        evaluate_report['mAP'].append(evaluate_data[classname]['mAP'])
        
        evaluate_report['avg_precision'].append(np.mean(evaluate_data[classname]['precisions']))
        evaluate_report['avg_recall'].append(np.mean(evaluate_data[classname]['recalls']))
        
    def clean_cols(col):        
        return ';'.join(str(e) for e in evaluate_report[col])
    
        
    
#     for k in evaluate_report.keys():
#         evaluate_report[k] = clean_cols(k)
    evaluate_report['classname'] = clean_cols('classname')
    evaluate_report['gt_total_annotation'] = clean_cols('gt_total_annotation')
    evaluate_report['pred_match_total_annotation'] = clean_cols('pred_match_total_annotation')
    evaluate_report['mAP'] = clean_cols('mAP')
    evaluate_report['avg_precision'] = clean_cols('avg_precision')
    evaluate_report['avg_recall'] = clean_cols('avg_recall')
    
  
    return evaluate_report


In [15]:
save_plots = True



def clean_iou(iou):
    return str("{:f}".format(iou)).replace('.','')[:3]


# json_file_path = '/mnt/vtq-69/logs/mask_rcnn/100519_112427-evaluate_050-hmd-train-160519_183815/'
# json_file_name = 'classification_rpt-train.json'

# evaluate_plots_dir = "evaluation_hmd_20190329T001052"
# plots_path = "/home/nikhil/Documents/ai-ml-dl-data/logs/mask_rcnn"

username = "alpha"
# sysip = '69'
base_path = "/mnt/remotemnt/vtd-69/aimldl-logs/mask_rcnn"
base_path = "/mnt/remotemnt/vtd-69/aimldl-logs/mask_rcnn/bkup"

##---
iou = 0.90
dataset_id = '130519_185650'
exec_on_timestamp='220519_163838'
subset = 'test'

## 100519_112427-evaluate_050-hmd-train-270519_135830

##---
iou = 0.50
dataset_id = '140519_193751'
exec_on_timestamp='270519_150700'
subset = 'train'

iou = 0.50
dataset_id = '130519_185650'
exec_on_timestamp='270519_150700'
subset = 'train'

##---
dbname='hmd'

##---
evaluate_dir = dataset_id+'-evaluate_'+clean_iou(iou)+'-'+dbname+'-'+subset+'-'+exec_on_timestamp
print("evaluate_dir: {}".format(evaluate_dir))

evaluate_data_log_path = os.path.join(base_path, evaluate_dir)
json_file='classification_rpt-'+subset+'.json'

evaluate_reports_path = os.path.join(base_path, 'evaluate_reports')


evaluate_plots_path = os.path.join(base_path, evaluate_dir, 'plots')
print("evaluate_plots_path: {}".format(evaluate_plots_path))

evaluate_dir = dataset_id+'-evaluate_'+'*-'+dbname+'-'+subset+'-*'

classification_rpt_files = glob.glob(os.path.join(base_path, evaluate_dir, json_file))
print("classification_rpt_files: {}".format(classification_rpt_files))




evaluate_dir: 130519_185650-evaluate_050-hmd-train-270519_150700
evaluate_plots_path: /mnt/remotemnt/vtd-69/aimldl-logs/mask_rcnn/bkup/130519_185650-evaluate_050-hmd-train-270519_150700/plots
classification_rpt_files: ['/mnt/remotemnt/vtd-69/aimldl-logs/mask_rcnn/bkup/130519_185650-evaluate_090-hmd-train-220519_161249/classification_rpt-train.json', '/mnt/remotemnt/vtd-69/aimldl-logs/mask_rcnn/bkup/130519_185650-evaluate_090-hmd-train-220519_180553/classification_rpt-train.json', '/mnt/remotemnt/vtd-69/aimldl-logs/mask_rcnn/bkup/130519_185650-evaluate_090-hmd-train-220519_164124/classification_rpt-train.json', '/mnt/remotemnt/vtd-69/aimldl-logs/mask_rcnn/bkup/130519_185650-evaluate_090-hmd-train-220519_171004/classification_rpt-train.json', '/mnt/remotemnt/vtd-69/aimldl-logs/mask_rcnn/bkup/130519_185650-evaluate_090-hmd-train-220519_173744/classification_rpt-train.json']


In [16]:
## create evaluate_reports_path
common.mkdir_p(evaluate_reports_path)
if save_plots:
    # print(evaluate_plots_path)
    common.mkdir_p(evaluate_plots_path)


In [22]:
timestamp = ("{:%d%m%y_%H%M%S}").format(datetime.datetime.now())
evaluate_report_filepath = os.path.join(evaluate_reports_path,'evalaute_report-'+timestamp+'.csv')
print(evaluate_report_filepath)

evaluate_stats_summary=[]
for evaluate_data_filepath in classification_rpt_files:
    # evaluate_data_filepath=os.path.join(evaluate_data_log_path, json_file)
    evaluate_json_data=load_evaluate_data(evaluate_data_filepath)
    # print(evaluate_json_data)
    print(evaluate_data_filepath)
    
    evaluate_dir_ref = evaluate_data_filepath.split(os.path.sep)[-2]
    print("evaluate_dir_ref: {}".format(evaluate_dir_ref))
    
    iou_ref = evaluate_dir_ref.split('-')[1].split('_')[1]
    evaluate_stats = get_stats(evaluate_json_data, username, iou_ref, evaluate_dir=evaluate_dir_ref, on=subset)
    print("evaluate_stats: {}".format(evaluate_stats))
    evaluate_stats_summary.append(evaluate_stats)

/mnt/remotemnt/vtd-69/aimldl-logs/mask_rcnn/bkup/evaluate_reports/evalaute_report-270519_180046.csv
/mnt/remotemnt/vtd-69/aimldl-logs/mask_rcnn/bkup/130519_185650-evaluate_090-hmd-train-220519_161249/classification_rpt-train.json
evaluate_dir_ref: 130519_185650-evaluate_090-hmd-train-220519_161249
classname: BG
classname: lane_marking
evaluate_stats: {'username': 'alpha', 'iou': '090', 'evaluate_dir': '130519_185650-evaluate_090-hmd-train-220519_161249', 'on': 'train', 'classname': 'lane_marking', 'gt_total_annotation': '4240', 'pred_match_total_annotation': '0', 'avg_precision': 'nan', 'avg_recall': 'nan', 'mAP': '0.0'}
/mnt/remotemnt/vtd-69/aimldl-logs/mask_rcnn/bkup/130519_185650-evaluate_090-hmd-train-220519_180553/classification_rpt-train.json
evaluate_dir_ref: 130519_185650-evaluate_090-hmd-train-220519_180553
classname: BG
classname: lane_marking
evaluate_stats: {'username': 'alpha', 'iou': '090', 'evaluate_dir': '130519_185650-evaluate_090-hmd-train-220519_180553', 'on': 'train

In [23]:
## Dump to a csv file
df = pd.DataFrame.from_dict(evaluate_stats_summary)
# df.fillna(0)
df = df.replace('nan', 0)
df.to_csv(evaluate_report_filepath, index=False )

In [24]:
# print(evaluate_stats_summary)
print(df)

   avg_precision  avg_recall     classname  \
0              0           0  lane_marking   
1              0           0  lane_marking   
2              0           0  lane_marking   
3              0           0  lane_marking   
4              0           0  lane_marking   

                                        evaluate_dir gt_total_annotation  iou  \
0  130519_185650-evaluate_090-hmd-train-220519_16...                4240  090   
1  130519_185650-evaluate_090-hmd-train-220519_18...                4240  090   
2  130519_185650-evaluate_090-hmd-train-220519_16...                4240  090   
3  130519_185650-evaluate_090-hmd-train-220519_17...                4240  090   
4  130519_185650-evaluate_090-hmd-train-220519_17...                4240  090   

   mAP     on pred_match_total_annotation username  
0  0.0  train                           0    alpha  
1  0.0  train                           0    alpha  
2  0.0  train                           0    alpha  
3  0.0  train           

In [ ]:
for stats in evaluate_stats_summary:
#     print("stats['gt_total_annotation'] : {}".format(stats['gt_total_annotation'] ))
#     print("stats: {}".format(stats))
#     print("gt_total_annotation: {}".format(type(gt_total_annotation)))
    
#     for e in stats['gt_total_annotation']:
#         print(e)
    
            
# #     def str_to_int(col):  
# #         for e in stats[col]:
# #             return int(e)
    
# #     stats['gt_total_annotation'] = str_to_int('gt_total_annotation')
# #     stats['pred_match_total_annotation'] = str_to_int('pred_match_total_annotation')
    
# #     stats['gt_total_annotation']=int(stats['gt_total_annotation'])
# #     stats['pred_match_total_annotation']=int(stats['pred_match_total_annotation'])
# #     stats['mAP']=float(stats['mAP'])
   
    
    # pred_match_total_annotation sorting
    iArr = {}
    for i in range(len(stats["gt_total_annotation"])):
      iArr[stats["gt_total_annotation"][i]] = {"classname":stats["classname"][i],"gt_total_annotation":stats["gt_total_annotation"][i],"pred_match_total_annotation":stats["pred_match_total_annotation"][i], "mAP":stats["mAP"][i]}
    sortedVal = sorted(iArr,reverse=True)

    sorted_ann = {"classname":[],"gt_total_annotation":[],"pred_match_total_annotation":[],"mAP":[]}
    for i in range(len(sortedVal)):
      sorted_ann["classname"].append(iArr[sortedVal[i]]["classname"])
      sorted_ann["gt_total_annotation"].append(iArr[sortedVal[i]]["gt_total_annotation"])
      sorted_ann["pred_match_total_annotation"].append(iArr[sortedVal[i]]["pred_match_total_annotation"])
      sorted_ann["mAP"].append(iArr[sortedVal[i]]["mAP"])


    print(sorted_ann)

In [ ]:
def str_to_int(col):
    return int(str(e) for e in evaluate_stats_summary[col])
def str_to_float(col):
    return float(str(e) for e in evaluate_stats_summary[col])

In [ ]:
for i in evaluate_stats_summary: 
    i['gt_total_annotation']=int(i['gt_total_annotation'])
    i['pred_match_total_annotation']=int(i['pred_match_total_annotation'])
    i['mAP']=float(i['mAP'])
   
    

In [ ]:
for i in evaluate_stats_summary: 
    print(type(i['mAP']))


In [ ]:
def sort_evaluate_data(stats, sort_on, f1, f2):
 
    
    
    iArr = {}
    assort = { "classname":[] }
    assort[sort_on] = []
    assort[f1] = []
    assort[f2] = []

    
    for i in range(len(stats[sort_on])):
      iArr[stats[sort_on][i]] = { "classname": stats["classname"][i] }
      iArr[stats[sort_on][i]][sort_on] = stats[sort_on][i]
      iArr[stats[sort_on][i]][f1] = stats[f1][i]
      iArr[stats[sort_on][i]][f2] = stats[f2][i]
  
    
    sorted_val = sorted(iArr,reverse=True)
    for i in range(len(sorted_val)):
      assort["classname"].append(iArr[sorted_val[i]]["classname"])
      assort[sort_on].append(iArr[sorted_val[i]][sort_on])
      assort[f1].append(iArr[sorted_val[i]][f1])
      assort[f2].append(iArr[sorted_val[i]][f2])

    return assort

In [ ]:
for stats in evaluate_stats:
    print("stats: {}".format(stats))
    
    ##TODO: sort the sort!
    ## gt_total_annotation sorting
    sort_on="gt_total_annotation"
    f1="pred_match_total_annotation"
    f2="mAP"
    sorted_gt_total_annotation = sort_evaluate_data(stats, sort_on, f1, f2)
    print("sorted_gt_total_annotation: {}".format(sorted_gt_total_annotation))

#     ## pred_match_total_annotation sorting
#     f1="gt_total_annotation"
#     sort_on="pred_match_total_annotation"
#     f2="mAP"

#     sorted_pred_match_total_annotation = sort_evaluate_data(stats, sort_on, f1, f2)
#     print("sorted_pred_match_total_annotation: {}".format(sorted_pred_match_total_annotation))

#     ## mAP sorting
#     f1="gt_total_annotation"
#     f2="pred_match_total_annotation"
#     sort_on="mAP"
#     sorted_mAP = sort_evaluate_data(stats, sort_on, f1, f2)
#     print("sorted_mAP: {}".format(sorted_mAP))

#     sorted_pred_match_total_annotation = stats['pred_match_total_annotation']
#     sorted_gt_total_annotation = stats['gt_total_annotation']
#     sorted_mAP = stats['mAP']
    
    ## plot_for gt_total_annotation
    legend_label='Total Ground Truth Annotation'
    plot_title='Total Annotations per Category over HMD Training Dataset'
    scale=20000
    offset=700
    draw_and_save_line_graph(sorted_gt_total_annotation,sorted_gt_total_annotation['gt_total_annotation'],legend_label,plot_title,scale,offset)

#     ## plot_for pred_match_total_annotation
#     legend_label='Total Predicted Annotation'
#     plot_title='Total Predicted Annotations per Category over HMD Training Dataset'
#     scale=12000
#     offset=400
#     draw_and_save_line_graph(sorted_pred_match_total_annotation,sorted_pred_match_total_annotation['pred_match_total_annotation'],legend_label,plot_title,scale,offset)

#     ## Read saved plot image and display here using matplotlib
#     img=mpimg.imread(save_file)
#     imgplot = plt.imshow(img)
#     plt.axis('off')
#     plt.show()

#     ## Read saved plot image and display here using skimage
#     skimage.io.imshow(save_file)

#     ## Rounding off the mAP values 
#     rounded_mAP = []
#     for i in range(len(sorted_mAP['mAP'])):
#         rounded_mAP.append(round(sorted_mAP['mAP'][i],2))
#     print(rounded_mAP)

#     ## plot_for mAP
#     legend_label='mAP (mean Average Precision)'
#     plot_title='Mean Average Precision per Category over HMD Training Dataset'
#     scale=1
#     offset=0.16
#     draw_and_save_line_graph(sorted_mAP,rounded_mAP,legend_label,plot_title,scale,offset)

#     gt_label='Total Ground Truth Annotation'
#     pred_label='Total Predicted Annotation'
#     mAP_label='mAP (Mean Avg. Precision)'
#     plot_title='Total Annotations & mAP per Category over HMD Training Dataset'

#     ##
#     def plot_and_save_pred_stats(sorted_gt_total_annotation, gt_label, pred_label, mAP_label, plot_title):
#         labels=sorted_gt_total_annotation['classname']
#         x_pos_labels = np.arange(len(labels))
#         gt_total_annotation=sorted_gt_total_annotation['gt_total_annotation']
#         pred_match_total_annotation=sorted_gt_total_annotation['pred_match_total_annotation']
#         mAP=sorted_gt_total_annotation['mAP']

#         plt.plot(x_pos_labels, gt_total_annotation, 's-' , alpha=0.7 , label=gt_label )
#         plt.plot(x_pos_labels, pred_match_total_annotation, 's-' , alpha=0.7 , label=pred_label,color='red')
#         plt.plot(x_pos_labels, mAP, 's-' , alpha=0.7 , label=mAP_label, color='green')

#         # plt.ylim(0,22000)
#         plt.xticks(x_pos_labels, labels, rotation=80, fontsize=10)
#         plt.title(plot_title)
#         plt.rcParams["figure.figsize"] = [22,15]
#         plt.legend(loc='upper right', prop={'size': 18})
#         # for a,b in zip(x_pos_labels, gt_total_annotation):
#         #     c=b+1250
#         #     plt.text(a, c, str(b),rotation='vertical',ha='center', fontsize=12)
#         # for a,b in zip(x_pos_labels, pred_match_total_annotation):
#         #     c=b+1250
#         #     plt.text(a, c, str(b),rotation='vertical',ha='center', fontsize=12)
#         # for a,b in zip(x_pos_labels, mAP):
#         #     c=b+1250
#         #     plt.text(a, c, str(b),rotation='vertical',ha='center', fontsize=12)
#         plot_file_name="Total Annotations & mAP per Category over HMD Training Dataset"
#     #     save_file = os.path.join(evaluate_plots_path, plot_file_name+".png")
#     #     plt.savefig(save_file, bbox_inches='tight', pad_inches=0.5 )

#     plot_and_save_pred_stats(sorted_gt_total_annotation, gt_label, pred_label, mAP_label, plot_title)


In [ ]:
for stats in evaluate_stats_summary:

    labels=stats['classname']
    gt_total_annotation=stats['gt_total_annotation']
    x_pos_labels = np.arange(len(labels))
    plt.plot(x_pos_labels, gt_total_annotation, 's-' ,label='Total Ground Truth Annotation', alpha=0.7)
    plt.ylim(0,400)
    plt.xticks(x_pos_labels, labels, rotation=80, fontsize=10)
    plt.title('Total Ground Truth Annotation per Category over HMD Training Dataset')
    plt.legend(loc='upper right', prop={'size': 18})
    plt.rcParams["figure.figsize"] = [22,15]  
    for a,b in zip(x_pos_labels, gt_total_annotation):
        c=b+10
        plt.text(a, c, str(b),rotation='vertical',ha='center')
#     save_file = os.path.join(filepath,"Total Ground Truth Annotation per Category over HMD Training Dataset"+".png")
#     plt.savefig(save_file, bbox_inches='tight' ,pad_inches=0.5 )

In [ ]:
for stats in evaluate_stats_summary:
    labels=stats['classname']
    avg_precision=stats['avg_precision']
    avg_recall=stats['avg_recall']
    mAP=stats['mAP']
    x_pos_labels = np.arange(len(labels))
    plt.plot(x_pos_labels, avg_precision, 's-' , alpha=0.7 , label='avg_precision' )
    plt.plot(x_pos_labels, avg_recall, 's-' , alpha=0.7 , label='avg_recall',color='red')
    plt.plot(x_pos_labels, mAP, 's-' , alpha=0.7 , label='mAP (Mean Avg. Precision)', color='green')

    plt.ylim(0,10)
    plt.xticks(x_pos_labels, labels, rotation=80, fontsize=18)
    plt.title('Total Annotations & mAP per Category over HMD Training Dataset')
    plt.rcParams["figure.figsize"] = [22,15]
    plt.legend(loc='upper right', prop={'size': 18})


In [ ]:
import matplotlib.pyplot as plt
x=[[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4]]
y=[[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4]]
for i in range(len(x)):
    plt.figure()
    plt.plot(x[i],y[i])